# Proyecto del Día 15 - Optimizar Modelos de Machine Learning

El proyecto del día de hoy consiste en tomar un código que ya está desarrollado con Scikit-Learn, y modificarlo agregando varios apspectos que hemos aprendido el día de hoy.

Este programa es un simple análisis usando el modelo de **Bosques Aleatorios** en un dataset de **Scikit-Learn** sobre registros médicos de **pacientes con diabetes**.

### Consigna

Tu trabajo aquí simplemente consiste en ejecutar este código, y analizarlo para identificar qué hace cada línea, y cuál es su función dentro del código. Una vez que te hayas asegurado de haber identificado sus elementos y de haber comprendido su funcionamiento en general, realiza todas las modificaciones que creas necesarias, para que este código incluya:

+ Preprocesamiento de datos con StandardScaler
+ Selección de mejores categorías (investiga cuál es la mejor función de cálculo)
+ Pipelines
+ Evaluación de modelo

Suena fácil, pero te aseguro que poner todo junto en un mismo proyecto, puede hacer que las cosas se compliquen un poco.

Te deseo lo mejor como siempre, esperando que este proyecto implique un buen tiempo de diversión y resolución de problemas.

In [17]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Cargar el dataset de diabetes
diabetes = datasets.load_diabetes()

# Convertir a DataFrame para facilitar el análisis exploratorio
diabetes_df = pd.DataFrame(data=np.c_[diabetes['data'], diabetes['target']],
                           columns=diabetes['feature_names'] + ['target'])

# Convertir 'target' en categorías para clasificación
diabetes_df['target'] = (diabetes_df['target'] > diabetes_df['target'].median()).astype(int)

# División de datos en conjuntos de entrenamiento y prueba
X = diabetes_df.drop('target', axis=1)
y = diabetes_df['target']
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo RandomForest
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_entrena, y_entrena)

# Realizar predicciones con el conjunto de prueba
predicciones = modelo.predict(X_prueba)

# Evaluación del modelo
puntaje = modelo.score(X_prueba, y_prueba)
print(f"\nPrecisión del modelo: {puntaje:.2f}")



Precisión del modelo: 0.72


### Código optimizado

In [41]:
# Copia y pega aquí el código anterior, y optimízalo según la consigna
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.pipeline import Pipeline

# Cargar el dataset de diabetes
diabetes = datasets.load_diabetes()

# Convertir a DataFrame para facilitar el análisis exploratorio
diabetes_df = pd.DataFrame(data=np.c_[diabetes['data'], diabetes['target']],
                           columns=diabetes['feature_names'] + ['target'])
# Convertir 'target' en categorías para clasificación
diabetes_df['target'] = (diabetes_df['target'] > diabetes_df['target'].median()).astype(int)

# División de datos en conjuntos de entrenamiento y prueba
X = diabetes_df.drop('target', axis=1)
y = diabetes_df['target']
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
selector = SelectKBest(score_func=f_classif) # Buscar las mejores características (¿Qué variables son las más importantes?)
scaler = StandardScaler() # Realizo el escalado de datos 
model = RandomForestClassifier(random_state=42) # creo el modelo
# NOTA cuando se usan los pipelines, ya no necesito ejecutar el fit_transform ni el transform
# X_entrena_selected = selector.fit_transform(X_entrena, y_entrena)
# X_prueba_selected = selector.transform(X_prueba)
# Esto se hace internamente en el pipeline.

In [43]:
pipeline = Pipeline([
    ("select", selector),
    ("scaler", scaler),
    ("model", model)
])

# Verificar los mejores parámetros para el modelo y además verificar que variables son las más importantes
param_grid = {
    'select__k': [3, 4, 5, 6, 7],  # número de características a seleccionar
    'model__n_estimators': [50, 100, 200],
    'model__max_features': ['sqrt', 'log2'],
    'model__max_depth': [4, 5, 6, 7, 8],
    'model__criterion': ['gini', 'entropy']
}

# En lugar de meter el modelo directamente en el GridSearch, meto todo el proceso (pipeline)
grid_search  = GridSearchCV(estimator=pipeline, 
                      param_grid=param_grid,
                      cv=5)

# Hago el fit
grid_search.fit(X_entrena, y_entrena)


# Predicción y evaluación
predicciones = grid_search.predict(X_prueba)
puntaje = grid_search.score(X_prueba, y_prueba)
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Precisión del modelo: {puntaje:.2f}")

# Indices de variables seleccionadas
selected_features = grid_search.best_estimator_.named_steps['select'].get_support(indices=True)
# Nombres de las variables seleccionadas
features_names = X.columns[selected_features]
print("Variables seleccionadas:", list(features_names))

Mejores parámetros: {'model__criterion': 'gini', 'model__max_depth': 5, 'model__max_features': 'sqrt', 'model__n_estimators': 100, 'select__k': 5}
Precisión del modelo: 0.78
Variables seleccionadas: ['bmi', 'bp', 's3', 's4', 's5']


C:\Users\Atmosfera\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [44]:
score_cross_val = cross_val_score(pipeline, X, y, cv=5)
print(f"Puntuación de validación cruzada = {score_cross_val}")
print(f"Promedio de validación cruzada = {np.mean(score_cross_val):.2f}")

Puntuación de validación cruzada = [0.69662921 0.75280899 0.64772727 0.67045455 0.65909091]
Promedio de validación cruzada = 0.69


In [33]:
df_filtrado = diabetes_df[diabetes_df.isna().any(axis=1)]

In [35]:
df_filtrado

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
